Q3 (scatterplot)
idea 1:
- scatterplot
- eix x: school
- eix y: mass shootings
- color: year
- point: state in year

idea2:
- violin plot
- violin: state
- distribucio violin: years

idea3:
- scatterplot de totals, sense separar per anys


county, state, shootings, school, year